In [1]:
from scraper.datamodel import DbApi

In [3]:
db = DbApi('../data/backup2.db')

In [4]:
db.conn.execute('select count(*) from comments').fetchall()

[(1459660,)]

In [6]:
db.conn.execute('select count(*) from submissions').fetchall()

[(222211,)]

In [7]:
db.conn.execute('select count(*) from sentences').fetchall()

[(5816712,)]

In [15]:
db.conn.execute('select src_is_subm, count(*) from sentences group by 1').fetchall()

[(0, 2908670), (1, 2908042)]

In [22]:
db.conn.execute('select lower(text), count(*) from sentences where src_is_subm=1 group by 1 order by 2 desc limit 50').fetchall()

[("i don't know what to do.", 3465),
 ("i don't know.", 1931),
 ('i want to die.', 1780),
 ('i hate myself.', 1146),
 ('i need help.', 1099),
 ("i don't know what to do anymore.", 1090),
 ('i just want to die.', 1088),
 ('what do i do?', 977),
 ('please help me.', 872),
 ('what should i do?', 697),
 ('thanks for reading.', 678),
 ('i have no friends.', 675),
 ("i can't do this anymore.", 670),
 ("i'm so tired.", 607),
 ("what's the point?", 546),
 ("i just don't know what to do.", 524),
 ('i need help', 488),
 ("i can't take it anymore.", 428),
 ('i give up.', 428),
 ('i want to kill myself.', 424),
 ('what can i do?', 421),
 ("i don't know what to do", 405),
 ('i hate my life.', 399),
 ('i hate it.', 396),
 ("i don't want to die.", 395),
 ('i want to die', 363),
 ('i have nothing.', 340),
 ("i just don't know what to do anymore.", 332),
 ('please help me', 331),
 ("i can't do it anymore.", 328),
 ("i don't know why.", 316),
 ("i don't want to live anymore.", 315),
 ('i feel so alone.'

In [23]:
qry_nobots = """
with bots as (
    select author, count(*)
    from comments 
    group by 1
    having count(*) >=20
        union all
    select 'SeriouslyLucifer', -1
        union all
    select 'peadditer', -1
)
select lower(text), count(*) 
from sentences s
join comments c
  on s.src_id = c.id
  and src_is_subm=0 
left join bots b
  on c.author = b.author
where b.author is null
and c.author <> '[deleted]'
group by 1 order by 2 desc limit 50
"""
db.conn.execute(qry_nobots).fetchall()

[('thank you so much!', 355),
 ('username checks out', 210),
 ('i love it.', 205),
 ("i don't know.", 178),
 ('what do you mean?', 174),
 ('i like it.', 165),
 ('thank you :)', 163),
 ('thanks so much!', 142),
 ('happy cake day!', 132),
 ('i love it!', 125),
 ('not at all.', 121),
 ('thank you so much.', 121),
 ('thanks for sharing.', 116),
 ('thank you very much!', 113),
 ('best of luck!', 112),
 ('what are you talking about?', 111),
 ('thanks for the info.', 106),
 ("i don't get it.", 104),
 ("i don't think so.", 103),
 ('thanks for the reply.', 103),
 ('thanks a lot!', 98),
 ('thanks for the info!', 98),
 ('i agree with you.', 95),
 ('thanks for sharing!', 93),
 ('username checks out.', 92),
 ('that makes sense.', 90),
 ('( ͡° ͜ʖ ͡°)', 89),
 ('hope this helps!', 89),
 ('good to know.', 84),
 ('thank you for your submission!', 84),
 ('hope this helps.', 82),
 ('keep it up!', 81),
 ('thanks for the advice.', 79),
 ('enter pls bot', 76),
 ('i get it.', 75),
 ('yes it is.', 73),
 ('what

In [17]:
# These fucking guys.
qry_lowfreq_bots = """
select author, subreddit, s.text, count(*), max(c.body)
from comments c
join sentences s
  on  s.src_id = c.id
  and s.src_is_subm = 0
where s.text = ? or s.text = ? 
order by 1, 3, 4 desc, 2
"""
bad_sents = ['___\n\n[**Here are our subreddit rules.', "**Unfortunately, we've had to remove your post."]

db.conn.execute(qry_lowfreq_bots, bad_sents).fetchall()

[('SeriouslyLucifer',
  'FortniteCompetitive',
  "**Unfortunately, we've had to remove your post.",
  190,
  "**Unfortunately, we've had to remove your post.** \n\n___\n\n###Posts Should Be High Quality And Encourage Discussion\n\nWe want /r/FortniteCompetitive to be the best source of discussion for the game online - and to do that we have to ensure that quality content isn’t drowned out by low effort or low-quality submissions.  Content must be related to Fortnite. If your post contains a suggestion for the game, it must contain Fortnite elements.\n\n**High Quality Guidelines:**\n\n- Posts about similar games are not allowed.\n\n- Every post should be a space for meaningful discussion. While this doesn't mean every post needs to be a discursive essay, posts that don't offer room for meaningful discussion will be removed.\n\n- Screenshots are removed at moderation discretion if they do not contain sufficient supporting text to encourage a discussion.\n\n- Videos/images taken with a ph

In [19]:
# These fucking guys.
qry_lowfreq_bots = """
select author, subreddit, s.text, count(*), max(c.body)
from comments c
join sentences s
  on  s.src_id = c.id
  and s.src_is_subm = 0
where s.text = 'Thank you for your submission!'
order by 1, 3, 4 desc, 2
"""

db.conn.execute(qry_lowfreq_bots).fetchall()

[('peadditer',
  'CoDCompetitive',
  'Thank you for your submission!',
  87,
  "Thank you for your submission! Unfortunately, your submission has been removed for the following reason(s):\n\nPlease use the 'Daily Discussion Thread' stickied at the top of the subreddit for your question/topic. Thanks!\n\n\n\nIf you have any questions regarding the ruleset of /r/CoDCompetitive, please refer to the [FAQ](/r/CoDCompetitive/wiki/index) or [message the moderators](/message/compose?to=%2Fr%2FCoDCompetitive). Thank you!")]